## Trying pyspark
we can install Apache Spark on a local Windows Machine in a pseudo-distributed mode (managed by Spark’s standalone cluster manager) and run it using PySpark

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf()
conf.setMaster("local").setAppName("cr_pred")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
print("Current Spark version is : {0}".format(spark.version))

Current Spark version is : 3.1.1


In [3]:
pandas_df = pd.read_pickle("training_df")

In [4]:
df = spark.createDataFrame(pandas_df)

In [ ]:
df.show(1)

Spark ML’s algorithms expect the data to be represented in two columns: Features and Labels. Features is an array of data points of all the features to be used for prediction. Labels contain the output label for each data point.

In [ ]:
feature_columns = df.columns[1:] # here we omit the first column --> our label

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

In [ ]:
data = assembler.transform(df)

In [ ]:
data.show(1)

In [ ]:
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
algo1 = LinearRegression(featuresCol="features", labelCol="cr")
model_1 = algo1.fit(train)

In [ ]:
def eval_model(model, test):
    start = time()
    evaluation_summary = model.evaluate(test)
    end = time()
    result = end - start
    print('Training time = %.3f seconds' % result)
    return evaluation_summary

In [ ]:
def get_eval_metrics(summary):
    summary.meanAbsoluteError
    summary.rootMeanSquaredError
    summary.r2

In [ ]:
summary1 = eval_model(model_1, test)
get_eval_metrics(summary1)

In [ ]:
algo2 = RandomForestRegressor(featuresCol="indexedFeatures")
model_2 = algo2.fit(train)

In [ ]:
summary2 = eval_model(model_2, test)
get_eval_metrics(summary2)

In [ ]:
spark.stop()

# References
https://github.com/apache/spark/blob/master/examples/src/main/python/ml/random_forest_regressor_example.py